<a href="https://colab.research.google.com/github/Diljitsingh14/Colab-Notebooks/blob/main/Fine_Tune_HF_LLM_GPT2_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torchvision
!pip install transformers

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [4]:
# Load pre-trained model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [9]:
from PyPDF2 import PdfReader

pdfreader = PdfReader('budget_speech.pdf')
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [10]:
with open("speech_text.txt",'w') as f:
  f.write(raw_text)

In [5]:
# Load your dataset
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/content/speech_text.txt",
    block_size=128  # Adjust according to your needs
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to True if you're using a masked language model
)

In [8]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine-tuned-gpt2",
    overwrite_output_dir=True,
    num_train_epochs=10,  # Adjust as needed
    per_device_train_batch_size=4,  # Adjust as needed
    save_steps=10_000,  # Adjust as needed
    save_total_limit=2,
)

In [9]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Start training
trainer.train()

Step,Training Loss


TrainOutput(global_step=450, training_loss=2.4245675998263887, metrics={'train_runtime': 77.5599, 'train_samples_per_second': 23.208, 'train_steps_per_second': 5.802, 'total_flos': 117581414400000.0, 'train_loss': 2.4245675998263887, 'epoch': 10.0})

In [10]:
# Evaluate the model (optional)
results = trainer.evaluate()

# Save the fine-tuned model
trainer.save_model("./fine-tuned-gpt2")


ValueError: ignored